In [1]:
import pandas as pd
import numpy as np
from recordlinkage.preprocessing import clean
from recordlinkage.preprocessing import phonetic
from recordlinkage.index import Full
from recordlinkage.index import Block
import recordlinkage as rl

In [2]:
main_org = pd.read_excel('Main Records.xlsx')
main = main_org.copy()

In [3]:
dup_org = pd.read_excel('Duplicates.xlsx')
dup=dup_org.copy()

In [ ]:
#Sampling
'''
sample_num = 1000
main = main.sample(sample_num)
main = main.reset_index(drop = True)
dup = dup.sample(sample_num/2)
dup = dup.reset_index(drop=True)
'''

In [4]:
main.head()

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,1f27cdd1,Schwartz,Daniel,12/09/2013,NaN,NaN,Pentecostalism and other Charismatic,NaN,White,NaN,NaN,English
1,3bb0e015,Wishart,Ray,07/24/2007,NaN,NaN,Sikhism,NaN,Southeast Asian,NaN,NaN,NaN
2,4b532e78,Woods,Shirley,07/10/2020,NaN,NaN,United Church,NaN,East Asian,NaN,NaN,Korean
3,612a0deb,Germon,Tom,09/06/2014,NaN,Indian,NaN,Indian,Indian,Canada,Canada,NaN
4,5c1ac840,Williams,Betty,04/28/2020,NaN,NaN,NaN,NaN,Black,Canada,Canada,English


In [ ]:
#Run this if you want to run any of the commented lines below
'''
race = main['Race']
race_ind = race.isnull()
eth = main['Ethnicity']
eth_ind = eth.isnull()
'''

In [ ]:
'''
for i in range(0, race.shape[0]):
    if ((race[i] != eth[i]) and (race_ind[i] == False) and (eth_ind[i]==False)) :
        print ('Index = ', i, ", Race: ", race[i], ", Ethnicity: ", eth_ind[i])
'''
#Ethnicity == Race found

In [ ]:
'''
races=np.unique(main['Race'].dropna())
countries = np.unique(main['Citizenship'].dropna())
langs = np.unique(main['Language (Mother Tongue)'].dropna())
places = np.unique(main['Place of Birth'].dropna())
reli = np.unique(main['Religion'].dropna())
ind = np.unique(main['Indigenous Identity'].dropna())
'''

In [ ]:
'''
indig = main['Indigenous Identity']
indig_ind = indig.isnull()
race = main['Race']
race_ind = race.isnull()
for i in range(0, race.shape[0]):
    if ((race[i] != indig[i]) and (race_ind[i] == False) and (indig_ind[i]==False)) :
        print ('Index = ', i, ", Race: ", race[i], ", Indigenous Identity: ", indig[i])
'''
#Race = Indigenous Identity

In [ ]:
'''
indig = main['Indigenous Identity']
indig_ind = indig.isnull()
citizen = main['Citizenship']
citizen_ind = citizen.isnull()
for i in range(0, race.shape[0]):
    if(indig[i] == 'Alaska Native'):
        if ((citizen[i] != 'Canada') and (citizen_ind[i] == False) and (indig_ind[i]==False)) :
            print ('Index = ', i, ", Citizenship: ", citizen[i], ", Indigenous Identity: ", indig[i])
'''
#Alaskan Native means Canadian

In [ ]:
'''
for i in range(len(countries)):
    con = main[main['Citizenship'] == countries[i]]
    print('Unique Races in ', countries[i], ':\n',np.unique(con['Race'].dropna()) )
'''

In [ ]:
'''
for i in range(len(countries)):
    con = main[main['Citizenship'] == countries[i]]
    print('Unique languages in ', countries[i], ':\n',np.unique(con['Language (Mother Tongue)'].dropna()))
'''

In [ ]:
'''
for i in range(len(countries)):
    con = main[main['Citizenship'] == countries[i]]
    print('Unique places of birth in ', countries[i], ':\n',np.unique(con['Place of Birth'].dropna()))
'''
#If not a citizen of Canada, birth place = Citizenship

In [ ]:
'''
for i in range(len(places)):
    con = main[main['Place of Birth'] == places[i]]
    print('Unique Citizenships depending on POB: ', places[i], ':\n',np.unique(con['Citizenship'].dropna()))
'''

In [ ]:
'''
for i in range(len(races)):
    con = main[main['Race'] == races[i]]
    print('Unique Countries for race: ', race[i], ':\n',np.unique(con['Citizenship'].dropna()))
'''

In [ ]:
'''
for i in range(len(races)):
    con = main[main['Race'] == races[i]]
    print('Unique religions for race: ', races[i], ':\n',np.unique(con['Religion'].dropna()))
'''

In [ ]:
'''
for i in range(len(races)):
    con = main[main['Race'] == races[i]]
    print('Unique countries for race: ', races[i], ':\n',np.unique(con['Citizenship'].dropna()))
'''
# If race = 'Alaska Native', 'American Indian', 'Native Hawaiian','Pacific Islander' -> Citizen = Canada

In [ ]:
'''
for i in range(len(reli)):
    con = main[main['Religion'] == reli[i]]
    print('Unique races for religion: ', reli[i], ':\n',np.unique(con['Race'].dropna()))
'''

In [ ]:
'''
for i in range(len(countries)):
    con = main[main['Citizenship'] == countries[i]]
    print('Unique religions in ', countries[i], ':\n',np.unique(con['Religion'].dropna()))
'''

In [5]:
#Function to fill in null values
def clean_file(df):
    #Ethnicity == Race found
    #Race = Indigenous Identity
    #If not a citizen of Canada, citizenship -> birth place
    # If race = 'Alaska Native', 'American Indian', 'Native Hawaiian','Pacific Islander' -> Citizen = Canada
    race_list = ['Alaska Native', 'American Indian', 'Native Hawaiian','Pacific Islander']
    r = df['Race']
    ig = df['Indigenous Identity']
    e = df['Ethnicity']
    c = df['Citizenship']
    p = df['Place of Birth']
    r_i = df['Race'].isna()
    e_i = df['Ethnicity'].isna()
    i_i = df['Indigenous Identity'].isna()
    c_i = df['Citizenship'].isna()
    p_i = df['Place of Birth'].isna()

    for i in range(df.shape[0]):
        if ((r_i[i] == True) or (i_i[i] == True) or (e_i[i] == True)):
            if ((r_i[i] == False) or (i_i[i] == False) or (e_i[i] == False)):
                if r_i[i] == False:
                    if e_i[i] == True:
                        e[i] = r[i]
                    if i_i[i] == True:
                        ig[i] = r[i]
                if e_i[i] == False:
                    if r_i[i] == True:
                        r[i] = e[i]
                    if i_i[i] == True:
                        ig[i] = e[i]
                if i_i[i] == False:
                    if r_i[i] == True:
                        r[i] = ig[i]
                    if e_i[i] == True:
                        e[i] = ig[i]

        if r_i[i] == False:
            if r_i[i] in race_list:
                if c_i[i] == True:
                    c[i] = 'Canada'

        if c_i[i] == False:
            if c_i[i] != 'Canada':
                if p_i[i] == True:
                    p[i] = c[i] 
                
    return r,ig,e,c,p

In [6]:
a,b,c,d,e = clean_file(main)
main['Race'] = a
main['Indigenous Identity'] = b
main['Ethnicity'] = c
main['Citizenship'] = d
main['Place of Birth'] = e 

In [7]:
main

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,1f27cdd1,Schwartz,Daniel,12/09/2013,NaN,White,Pentecostalism and other Charismatic,White,White,NaN,NaN,English
1,3bb0e015,Wishart,Ray,07/24/2007,NaN,Southeast Asian,Sikhism,Southeast Asian,Southeast Asian,NaN,NaN,NaN
2,4b532e78,Woods,Shirley,07/10/2020,NaN,East Asian,United Church,East Asian,East Asian,NaN,NaN,Korean
3,612a0deb,Germon,Tom,09/06/2014,NaN,Indian,NaN,Indian,Indian,Canada,Canada,NaN
4,5c1ac840,Williams,Betty,04/28/2020,NaN,Black,NaN,Black,Black,Canada,Canada,English
...,...,...,...,...,...,...,...,...,...,...,...,...
19996,43d9ceca,Fernandez,Matthew,08/07/2015,NaN,American Indian,NaN,American Indian,American Indian,Canada,Canada,Arabic
19997,56c09f74,Matthews,Shari,02/28/2015,NaN,White,Other/not recorded,White,White,Canada,Canada,NaN
19998,25e05eb1,Konty,Lisa,12/15/2022,NaN,Black,NaN,Black,Black,NaN,NaN,English
19999,81d18750,Knowlton,Harold,10/24/2018,NaN,Alaska Native,NaN,Alaska Native,Alaska Native,Canada,Canada,NaN


In [8]:
a,b,c,d,e = clean_file(dup)
dup['Race'] = a
dup['Indigenous Identity'] = b
dup['Ethnicity'] = c
dup['Citizenship'] = d
dup['Place of Birth'] = e 

C:\Users\hamza\AppData\Local\Temp\ipykernel_321408\1021505405.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[i] = e[i]


In [9]:
dup

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,73c2ab6a,NaN,Steven,01/13/2008,NaN,Indian,NaN,Indian,Indian,Canada,NaN,NaN
1,488f1063,NaN,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,NaN,NaN,NaN,NaN
2,331bea71,Yerian,Ellwood,08/18/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English
3,ac3aaf9c,NaN,Robin,08/25/2013,NaN,Indian,NaN,Indian,Indian,Canada,NaN,English
4,ae9c2ae4,NaN,Raymond,05/25/2017,NaN,NaN,Other Christianity,NaN,NaN,Canada,NaN,English
...,...,...,...,...,...,...,...,...,...,...,...,...
10135,08606dd8,Avila,NaN,06/24/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10136,8507e3f4,Fernandez,NaN,08/07/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10137,d5285223,Mathews,NaN,02/28/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10138,7157d300,Konty,NaN,12/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
clean_main_last = clean (main["Last Name"])

In [11]:
clean_main_first = clean(main['First Name'])

In [12]:
clean_main_mother = clean(main["Mother's Birth Last Name"] )

In [13]:
main_draft = main.drop(['Last Name', 'First Name', "Mother's Birth Last Name"], axis = 1)

In [14]:
clean_main_last = phonetic(clean_main_last, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [15]:
clean_main_first = phonetic(clean_main_first, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [16]:
clean_main_mother = phonetic(clean_main_mother, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [17]:
clean_main = pd.concat([clean_main_last, clean_main_first, clean_main_mother, main_draft], axis = 1)

In [18]:
clean_main

,Last Name,First Name,Mother's Birth Last Name,ID,Date of Birth,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,S632,D540,NaN,1f27cdd1,12/09/2013,White,Pentecostalism and other Charismatic,White,White,NaN,NaN,English
1,W263,R000,NaN,3bb0e015,07/24/2007,Southeast Asian,Sikhism,Southeast Asian,Southeast Asian,NaN,NaN,NaN
2,W320,S640,NaN,4b532e78,07/10/2020,East Asian,United Church,East Asian,East Asian,NaN,NaN,Korean
3,G655,T500,NaN,612a0deb,09/06/2014,Indian,NaN,Indian,Indian,Canada,Canada,NaN
4,W452,B300,NaN,5c1ac840,04/28/2020,Black,NaN,Black,Black,Canada,Canada,English
...,...,...,...,...,...,...,...,...,...,...,...,...
19996,F655,M300,NaN,43d9ceca,08/07/2015,American Indian,NaN,American Indian,American Indian,Canada,Canada,Arabic
19997,M320,S600,NaN,56c09f74,02/28/2015,White,Other/not recorded,White,White,Canada,Canada,NaN
19998,K530,L200,NaN,25e05eb1,12/15/2022,Black,NaN,Black,Black,NaN,NaN,English
19999,K543,H643,NaN,81d18750,10/24/2018,Alaska Native,NaN,Alaska Native,Alaska Native,Canada,Canada,NaN


In [78]:
cm = clean_main[clean_main["Mother's Birth Last Name"].isna() == False]
cm[cm['Last Name'] != cm["Mother's Birth Last Name"] ]

,Last Name,First Name,Mother's Birth Last Name,ID,Date of Birth,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
175,M324,J160,M240,04bc59ab,09/27/2014,Pacific Islander,NaN,Pacific Islander,Pacific Islander,Canada,Canada,English
514,M324,B525,M240,b3b4b8ea,10/26/2007,Hispanic,Baptistism,Hispanic,Hispanic,Canada,Canada,Tagalog
1434,S436,L200,P243,ca34b0ea,05/24/2009,Black,Baptistism,Black,Black,NaN,NaN,English
1722,M324,J535,M240,fe291715,01/31/2015,Alaska Native,NaN,Alaska Native,Alaska Native,Canada,Canada,English
1839,P600,M650,F600,21cba746,07/18/2007,Southeast Asian,NaN,Southeast Asian,Southeast Asian,Canada,Canada,NaN
3185,K600,D100,C600,b9c0ce28,04/10/2008,American Indian,NaN,American Indian,American Indian,Canada,Canada,NaN
3351,K200,R200,C200,59f047b7,02/22/2007,Middle Easterner,Islam,Middle Easterner,Middle Easterner,NaN,NaN,NaN
3611,S462,E350,C462,397f9269,08/13/2010,American Indian,NaN,American Indian,American Indian,Canada,Canada,NaN
4095,M520,R534,M512,7a6db701,11/23/2014,Indian,NaN,Indian,Indian,Canada,Canada,NaN
4222,R262,C645,R326,d919c734,03/31/2007,Hispanic,Hinduism,Hispanic,Hispanic,Canada,Canada,Korean


In [19]:
clean_dup_last = clean (dup["Last Name"])

In [20]:
clean_dup_first = clean (dup["First Name"])

In [21]:
clean_dup_mother = clean (dup["Mother's Birth Last Name"])

In [22]:
dup_draft = dup.drop(['Last Name', 'First Name', "Mother's Birth Last Name"], axis = 1)

In [23]:
clean_dup_last = phonetic(clean_dup_last, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [24]:
clean_dup_first = phonetic(clean_dup_first, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [25]:
clean_dup_mother = phonetic(clean_dup_mother, method = 'soundex', concat=True, encoding='utf-8', decode_error='strict')

In [26]:
clean_dup = pd.concat([clean_dup_last, clean_dup_first, clean_dup_mother, dup_draft], axis = 1)

In [27]:
#Block Index
'''
indexer = rl.Index()
indexer.add(Block('First Name', 'First Name'))
indexer.add(Block('Last Name', 'Last Name'))
indexer.add(Block("Mother's Birth Last Name", "Mother's Birth Last Name"))
indexer.add(Block('Last Name', "Mother's Birth Last Name"))
indexer.add(Block( "Mother's Birth Last Name", 'Last Name'))
candidate_links = indexer.index(clean_main, clean_dup)
'''

In [138]:
#Full Index
indexer = rl.Index()
indexer.add(Full())
candidate_links = indexer.index(clean_main, clean_dup)

In [ ]:
'''
name_th = 0.9
compare_cl = rl.Compare()
compare_cl.string('Last Name', 'Last Name', method="levenshtein", threshold=name_th, label="Last Name")
compare_cl.string('First Name', 'First Name', method="levenshtein", threshold=name_th, label="First Name")
compare_cl.string("Mother's Birth Last Name", "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="Mother's Birth Last Name")
compare_cl.string("Mother's Birth Last Name", 'Last Name', method="levenshtein", threshold=name_th, label="Main-Mother->Last")
compare_cl.string('Last Name', "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="Main-Last->Mother")
compare_cl.exact("Race", "Race", label="Race")
compare_cl.exact("Citizenship", "Citizenship", label="Citizenship")
compare_cl.exact("Language (Mother Tongue)", "Language (Mother Tongue)", label="Language")

compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")
features = compare_cl.compute(candidate_links, clean_main,clean_dup)
'''

In [139]:
name_th = 0.9
compare_cl = rl.Compare()
compare_cl.exact('Last Name', 'Last Name',label="Last Name")
compare_cl.exact('First Name', 'First Name', label="First Name")
compare_cl.exact("Mother's Birth Last Name", "Mother's Birth Last Name", label="Mother's Birth Last Name")
compare_cl.exact("Mother's Birth Last Name", 'Last Name', label="Main-Mother->Last")
compare_cl.exact('Last Name', "Mother's Birth Last Name",  label="Main-Last->Mother")
compare_cl.exact("Race", "Race", label="Race")
compare_cl.exact("Religion", "Religion", label="Religion")
compare_cl.exact("Citizenship", "Citizenship", label="Citizenship")
compare_cl.exact("Language (Mother Tongue)", "Language (Mother Tongue)", label="Language")
compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")
features = compare_cl.compute(candidate_links, clean_main,clean_dup)

MemoryError: Unable to allocate 12.1 GiB for an array with shape (8, 202810140) and data type object

In [ ]:
#Weighted comparison
'''
name_th = 0.9

compare_cl = rl.Compare()
compare_cl.string('Last Name', 'Last Name', method="levenshtein", threshold=name_th, label="Last Name")
compare_cl.string('First Name', 'First Name', method="levenshtein", threshold=name_th, label="First Name")
compare_cl.string("Mother's Birth Last Name", "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="Mother's Birth Last Name")
compare_cl.string("Mother's Birth Last Name", 'Last Name', method="levenshtein", threshold=name_th, label="Main-Mother->Last")
compare_cl.string('Last Name', "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="Main-Last->Mother")
compare_cl.exact("Race", "Race", label="Race")
compare_cl.exact("Citizenship", "Citizenship", label="Citizenship")
compare_cl.exact("Language (Mother Tongue)", "Language (Mother Tongue)", label="Language")
compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")

compare_cl.string('Last Name', 'Last Name', method="levenshtein", threshold=name_th, label="1")
compare_cl.string('First Name', 'First Name', method="levenshtein", threshold=name_th, label="2")
compare_cl.string("Mother's Birth Last Name", "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="3")
compare_cl.string("Mother's Birth Last Name", 'Last Name', method="levenshtein", threshold=name_th, label="4")
compare_cl.string('Last Name', "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="5")
compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")

compare_cl.string('Last Name', 'Last Name', method="levenshtein", threshold=name_th, label="6")
compare_cl.string('First Name', 'First Name', method="levenshtein", threshold=name_th, label="7")
compare_cl.string("Mother's Birth Last Name", "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="8")
compare_cl.string("Mother's Birth Last Name", 'Last Name', method="levenshtein", threshold=name_th, label="9")
compare_cl.string('Last Name', "Mother's Birth Last Name", method="levenshtein", threshold=name_th, label="0")
compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")

features = compare_cl.compute(candidate_links, clean_main,clean_dup)
'''

In [ ]:
#oo many calcs
'''
features = features.reset_index()
weight_sum = []
for i in range(0, features.shape[0]):
    bw = (features.loc[i].sum() - features['level_0'].loc[i] - features['level_1'].loc[i])*3
    lw = (features['Race'].loc[i] + features['Citizenship'].loc[i] + features['Language'].loc[i]) * 2
    s = bw-lw
    weight_sum.append(s)
'''
    

In [92]:
found = features.copy()
found['sum'] = features.sum(axis = 1)
found = found[found['sum']> 1]
found = found.reset_index()

In [93]:
found

,level_0,level_1,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,sum
0,0,440,0,1,0,0,0,0,0,0,1,0,2
1,0,1085,0,1,0,0,0,0,0,0,1,0,2
2,0,1159,1,0,0,0,1,0,0,0,0,0,2
3,0,1165,0,1,0,0,0,0,0,0,1,0,2
4,0,1213,0,1,0,0,0,1,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305521,20000,6500,1,0,0,0,0,0,0,1,0,0,2
305522,20000,6904,1,0,0,0,0,0,0,1,1,0,3
305523,20000,7127,1,0,0,0,0,0,0,1,0,0,2
305524,20000,7508,1,0,0,0,0,0,0,1,0,0,2


In [117]:
weight_sum = []
l = found['First Name']
f = found['Last Name']
mn = found["Mother's Birth Last Name"]
e = found['Race']
r = found['Religion']
t = found['Language']
c = found['Citizenship']
d = found['DOB']
mnl = found["Main-Mother->Last"]
mnr = found["Main-Last->Mother"]
for i in range(0, found.shape[0]):
    x = 0
    y = 0
    if c[i] + r[i] + t[i] + e[i] == 4:
        x = 30
    elif c[i] + r[i] + t[i] + e[i] == 3:
        x = 25
    elif c[i] + r[i] + t[i] + e[i] == 2:
        x = 20
    elif c[i] + r[i] + t[i] + e[i] == 1:
        x = 10

    if f[i] + l[i] + d[i] + mn[i] == 4:
        y = 70
    elif f[i] + l[i] + d[i] == 3:
        y = 65
    elif f[i] + mn[i] + d[i] == 3:
        y = 60
    elif f[i] + mnl[i] + d[i] == 3:
        y = 55
    elif f[i] + mnr[i] + d[i] == 3:
        y = 55
    elif l[i] + mn[i] + d[i] == 3:
        y = 50
    elif f[i] + l[i] == 2:
        y = 50
    elif f[i] + mn[i]== 2:
        y = 40
    elif f[i] + mnl[i]== 2:
        y = 30
    elif f[i] + mnr[i]== 2:
        y = 30
    elif l[i] + mn[i]== 2:
        y = 30
    elif f[i] + d[i]== 2:
        y = 20
    elif d[i] + mn[i]== 2:
        y = 10

    weight_sum.append(x+y)

In [ ]:
'''
for i in range(0, found.shape[0]):
    bw = (found['Last Name'].loc[i] + found['First Name'].loc[i])*3
    lw = (found['Race'].loc[i] + found['Citizenship'].loc[i] + found['Language'].loc[i]) + found['Religion'].loc[i]) 
    if found["Mother's Birth Last Name"].loc[i] == 1:
        mw = 2
    elif ((found["Main-Mother->Last"].loc[i] == 1) or (found["Main-Last->Mother"].loc[i] == 1)):
        mw = 2
    else:
        mw = 0
    s = bw + lw + mw
    weight_sum.append(s)
'''

In [118]:
found['weight'] = weight_sum

In [119]:
found

,level_0,level_1,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,sum,weight
0,0,440,0,1,0,0,0,0,0,0,1,0,2,10
1,0,1085,0,1,0,0,0,0,0,0,1,0,2,10
2,0,1159,1,0,0,0,1,0,0,0,0,0,2,30
3,0,1165,0,1,0,0,0,0,0,0,1,0,2,10
4,0,1213,0,1,0,0,0,1,0,0,0,0,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305521,20000,6500,1,0,0,0,0,0,0,1,0,0,2,10
305522,20000,6904,1,0,0,0,0,0,0,1,1,0,3,20
305523,20000,7127,1,0,0,0,0,0,0,1,0,0,2,10
305524,20000,7508,1,0,0,0,0,0,0,1,0,0,2,10


In [132]:
match = found[found['weight']> 45]
print(len(match))

7726


In [133]:
#match.sort_values(by = ['sum'])
match.sort_values(by = ['sum'], ascending = False)

,level_0,level_1,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,sum,weight
90641,6032,539,1,1,1,1,1,1,1,0,1,1,9,95
155229,10329,1680,1,1,1,1,1,1,1,0,1,1,9,95
167354,11156,1913,1,1,1,1,1,1,1,0,1,1,9,95
254719,16680,6820,1,1,1,1,1,0,0,1,1,1,8,90
150846,10043,1606,1,1,1,1,1,1,0,0,1,1,8,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172376,11528,6730,1,1,0,0,0,0,0,0,0,0,2,50
172359,11528,1605,1,1,0,0,0,0,0,0,0,0,2,50
172029,11502,2998,1,1,0,0,0,0,0,0,0,0,2,50
172025,11501,6332,1,1,0,0,0,0,0,0,0,0,2,50


In [136]:
main_org.loc[13957]

ID                                 f2cda130
Last Name                             Mayer
First Name                          Melissa
Date of Birth                    07/14/2017
Mother's Birth Last Name                NaN
Indigenous Identity                     NaN
Religion                    Catholic Church
Ethnicity                               NaN
Race                                  White
Place of Birth                          NaN
Citizenship                          Canada
Language (Mother Tongue)           Mandarin
Name: 13957, dtype: object

In [137]:
dup_org.loc[1953]

ID                            ab7318f0
Last Name                        Moore
First Name                     Melissa
Date of Birth               04/09/2022
Mother's Birth Last Name           NaN
Indigenous Identity                NaN
Religion                           NaN
Ethnicity                          NaN
Race                               NaN
Place of Birth                     NaN
Citizenship                        NaN
Language (Mother Tongue)    Portuguese
Name: 1953, dtype: object

In [116]:
weight_sum = []
l = found['Last Name']
f = found['Last Name']
mn = found["Mother's Birth Last Name"]
e = found['Race']
r = found['Religion']
t = found['Language']
c = found['Citizenship']
d = found['DOB']
mnl = found["Main-Mother->Last"]
mnr = found["Main-Last->Mother"]
i = 305522
x = 0
y = 0
if c[i] + r[i] + t[i] + e[i] == 4:
    x = 30
elif c[i] + r[i] + t[i] + e[i] == 3:
    x = 25
elif c[i] + r[i] + t[i] + e[i] == 2:
    x = 20
elif c[i] + r[i] + t[i] + e[i] == 1:
    x = 10

if f[i] + l[i] + d[i] + mn[i] == 4:
    y = 70
elif f[i] + l[i] + d[i] == 3:
    y = 65
elif f[i] + mn[i] + d[i] == 3:
    y = 60
elif f[i] + mnl[i] + d[i] == 3:
    y = 55
elif f[i] + mnr[i] + d[i] == 3:
    y = 55
elif l[i] + mn[i] + d[i] == 3:
    y = 50
elif f[i] + l[i] == 2:
    y = 50
elif f[i] + mn[i]== 2:
    y = 40
elif f[i] + mnl[i]== 2:
    y = 30
elif f[i] + mnr[i]== 2:
    y = 30
elif l[i] + mn[i]== 2:
    y = 30
elif f[i] + d[i]== 2:
    y = 20
elif d[i] + mn[i]== 2:
    y = 10
f[i]

1